<a href="https://colab.research.google.com/github/sabyasm/ipython-notebooks/blob/master/Accelerating_Dual_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install plotly
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
 
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
import pandas as pd
import numpy as np

# Get Historical Data from Yahoo Finance

In [ ]:
df =  yf.download(["SPY","VSS","TLT","VXUS","BND","VSMGX"], start="2015-12-31") #can pass end=
df

[*********************100%***********************]  6 of 6 completed


Adj Close                          ... Volume                  
                  BND         SPY         TLT  ...  VSMGX      VSS     VXUS
Date                                           ...                         
2015-12-31  70.097244  184.496979  107.236542  ...      0   298200   922700
2016-01-04  70.062523  181.917801  108.010269  ...      0   586400  1497200
2016-01-05  70.149330  182.225494  107.574509  ...      0   246800   651100
2016-01-06  70.461830  179.926880  109.024132  ...      0   448200   991500
2016-01-07  70.470451  175.610184  109.219772  ...      0  1357400  1900800
...               ...         ...         ...  ...    ...      ...      ...
2021-04-26  85.389999  417.609985  139.860001  ...      0   157100  2343300
2021-04-27  85.169998  417.519989  138.639999  ...      0   179200  2664800
2021-04-28  85.209999  417.399994  138.729996  ...      0   294500  2501100
2021-04-29  85.139999  420.059998  138.320007  ...      0   278600  2289000
2021-04-30  85.260002  417.299988  138.639999  ...      0   181800  2948000

[1342 rows x 36 columns]

In [ ]:
def fig_from_df(df):
    fig = go.Figure()
    for col in df.columns:
      fig.add_trace(go.Scatter(x=df.index,y=df[col],name=col))
    return fig

fig = fig_from_df(df['Adj Close']/df['Adj Close'].iloc[0])
fig.show()

# Do Calculations

In [ ]:
monthly = df.resample('BM').last()['Close']
mon1 = monthly.pct_change(periods=1)
mon2 = monthly.pct_change(periods=2)
mon3 = monthly.pct_change(periods=3)
mon5 = monthly.pct_change(periods=5)
mon6 = monthly.pct_change(periods=6)
mon12 = monthly.pct_change(periods=12)

score = mon1+mon3+mon6
score_next = mon2+mon5
score

def adm_asset(i):
    asset = 'SPY'
    if score['VSS'].iloc[i]>score['SPY'].iloc[i]:
        asset = 'VSS'
    if score[asset].iloc[i]<0:
        asset = 'TLT'
    return asset

def dm_asset(i):
    asset = 'SPY'
    if mon12['VXUS'].iloc[i]>mon12['SPY'].iloc[i]:
        asset = 'VXUS'
    if mon12[asset].iloc[i]<0:
        asset = 'BND'
    return asset


assets = [''] * 7
for i in range(7,len(score)):
    assets.append(adm_asset(i-1))

dm_assets = [''] * 13
for i in range(13,len(score)):
    dm_assets.append(dm_asset(i-1))    

d = {'ADM_Asset' : pd.Series(assets, 
                       index =score.index), 
      'DM_Asset' : pd.Series(dm_assets, 
                        index =score.index)} 
signals = pd.DataFrame(d) 
signals

,ADM_Asset,DM_Asset
Date,,
2015-12-31,,
2016-01-29,,
2016-02-29,,
2016-03-31,,
2016-04-29,,
...,...,...
2020-12-31,VSS,SPY
2021-01-29,VSS,SPY
2021-02-26,VSS,SPY


# Get Outputs
## Summary Table

In [ ]:
summary = pd.concat([score.iloc[-1],
                     mon1.iloc[-1],
                     mon3.iloc[-1],
                     mon6.iloc[-1],
                     score_next.iloc[-1],
                     mon2.iloc[-1],
                     mon5.iloc[-1],
                    ],axis=1)
summary.columns = ['Score',
                   '1 Month',
                   '3 Month',
                   '6 Month',
                   'Next Score',
                   '2 Month',
                   '5 Month']
st = summary.T[['SPY','VSS','TLT']]*100
print(adm_asset(score.shape[0]-1))
st.round(1)

SPY


,SPY,VSS,TLT
Score,45.8,45.5,-18.4
1 Month,5.3,4.3,2.4
3 Month,12.8,10.6,-8.8
6 Month,27.8,30.7,-12.0
Next Score,25.0,21.9,-16.5
2 Month,9.7,6.3,-3.1
5 Month,15.3,15.6,-13.4


In [ ]:
st_table = st.reset_index().round(2)
st_table.columns = ['Period','SPY','VSS','TLT']

fig = go.Figure(data=[go.Table(
    header=dict(values=list(st_table.columns),
                align='right'),
    cells=dict(values=[st_table.Period, st_table.SPY, st_table.VSS, st_table.TLT],
               align='right'))
])
fig.write_image("Acceleration-Dual-Momentum-Table.png")
fig.show()

## Plot of ADM Returns

### Get Data Organized
First figure out the daily returns of our assets.

In [ ]:
rets = df['Adj Close'].pct_change(periods=1)+1

#Add the Asset the Strategies Are In
def add_asset(asset,rets):
    rets[asset]=''
    for i in range(len(signals)):
        rets.loc[signals.index.strftime('%Y-%m')[i],asset] = signals[asset].iloc[i]
    rets = rets[:-1] #I'm not entirely sure why I have to throw away the final row
    return rets

rets = add_asset('ADM_Asset',rets)
rets = add_asset('DM_Asset',rets)
rets = rets[rets['DM_Asset']!='']


#Determine Daily Returns of the Asset ADM is In
def add_asset_rets(name,strat):
    col_name = strat+' in '+name
    asset_name = strat+'_Asset'
    rets[col_name] = np.nan
    rets.loc[rets[asset_name]==name,col_name] = rets[name]
    rets.loc[rets[asset_name]==name,strat] = rets[name]

#Add in ADM    
strat = 'ADM'
rets[strat] = np.nan    
add_asset_rets('SPY',strat)
add_asset_rets('VSS',strat)
add_asset_rets('TLT',strat)

#Add in Regular Dual Momentum    
strat = 'DM'
rets[strat] = np.nan    
add_asset_rets('SPY',strat)
add_asset_rets('VXUS',strat)
add_asset_rets('BND',strat)

rets

,BND,SPY,TLT,VSMGX,VSS,VXUS,ADM_Asset,DM_Asset,ADM,ADM in SPY,ADM in VSS,ADM in TLT,DM,DM in SPY,DM in VXUS,DM in BND
Date,,,,,,,,,,,,,,,,
2017-01-03,1.000124,1.007650,1.004281,1.004555,1.007976,1.007193,SPY,SPY,1.007650,1.007650,NaN,NaN,1.007650,1.007650,NaN,NaN
2017-01-04,1.000742,1.005949,1.003845,1.005359,1.012556,1.011036,SPY,SPY,1.005949,1.005949,NaN,NaN,1.005949,1.005949,NaN,NaN
2017-01-05,1.005071,0.999205,1.015654,1.002870,1.011566,1.007492,SPY,SPY,0.999205,0.999205,NaN,NaN,0.999205,0.999205,NaN,NaN
2017-01-06,0.996063,1.003578,0.990818,0.999182,0.994335,0.997875,SPY,SPY,1.003578,1.003578,NaN,NaN,1.003578,1.003578,NaN,NaN
2017-01-09,1.002471,0.996699,1.008026,0.998772,0.996685,0.998084,SPY,SPY,0.996699,0.996699,NaN,NaN,0.996699,0.996699,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-22,1.000468,0.990867,1.004364,0.996381,0.995297,0.995824,VSS,SPY,0.995297,NaN,0.995297,NaN,0.990867,0.990867,NaN,NaN
2021-04-23,1.000117,1.010842,0.997649,1.007264,1.012149,1.011337,VSS,SPY,1.012149,NaN,1.012149,NaN,1.010842,1.010842,NaN,NaN
2021-04-26,0.999298,1.002088,0.998572,1.001202,1.006891,1.001228,VSS,SPY,1.006891,NaN,1.006891,NaN,1.002088,1.002088,NaN,NaN


In [ ]:
rets_cum = rets.drop(['ADM_Asset','DM_Asset'],axis=1)
rets_cum = rets_cum.cumprod()
rets_cum['ADM_Asset'] = rets['ADM_Asset']
rets_cum['DM_Asset'] = rets['DM_Asset']
def add_cumprod(name,strat):
    col_name = strat+' in '+name
    asset_name = strat+'_Asset'
    rets_cum.loc[rets[asset_name]==name,col_name] = rets_cum[strat]

add_cumprod('SPY','ADM')
add_cumprod('VSS','ADM')
add_cumprod('TLT','ADM')

add_cumprod('SPY','DM')
add_cumprod('VXUS','DM')
add_cumprod('BND','DM')

rets_cum = rets_cum.drop(['ADM_Asset','DM_Asset'],axis=1)
rets_cum

,BND,SPY,TLT,VSMGX,VSS,VXUS,ADM,ADM in SPY,ADM in VSS,ADM in TLT,DM,DM in SPY,DM in VXUS,DM in BND
Date,,,,,,,,,,,,,,
2017-01-03,1.000124,1.007650,1.004281,1.004555,1.007976,1.007193,1.007650,1.007650,NaN,NaN,1.007650,1.007650,NaN,NaN
2017-01-04,1.000867,1.013645,1.008142,1.009938,1.020632,1.018308,1.013645,1.013645,NaN,NaN,1.013645,1.013645,NaN,NaN
2017-01-05,1.005942,1.012839,1.023923,1.012836,1.032437,1.025937,1.012839,1.012839,NaN,NaN,1.012839,1.012839,NaN,NaN
2017-01-06,1.001981,1.016463,1.014522,1.012008,1.026587,1.023758,1.016463,1.016463,NaN,NaN,1.016463,1.016463,NaN,NaN
2017-01-09,1.004456,1.013108,1.022664,1.010766,1.023184,1.021796,1.013108,1.013108,NaN,NaN,1.013108,1.013108,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-22,1.188900,1.995182,1.294001,1.496516,1.581230,1.576585,2.189084,NaN,2.189084,NaN,1.566877,1.566877,NaN,NaN
2021-04-23,1.189039,2.016815,1.290959,1.507387,1.600441,1.594459,2.215680,NaN,2.215680,NaN,1.583865,1.583865,NaN,NaN
2021-04-26,1.188204,2.021025,1.289116,1.509199,1.611470,1.596418,2.230948,NaN,2.230948,NaN,1.587172,1.587172,NaN,NaN


### Plot Primary Assets

In [ ]:
fig = fig_from_df(rets_cum[['SPY','VSS','TLT']])
fig.update_layout(
    title="Comparison of Primary Assets",
    xaxis_title="Date",
    yaxis_title="Return of $1 Invested")
fig.write_image("Acceleration-Dual-Momentum-Assets-Returns.png")
fig.show()

### Plot All

In [ ]:
fig = fig_from_df(rets_cum)
fig.update_layout(
    title="Comparison of All Asset & Strategy Returns",
    xaxis_title="Date",
    yaxis_title="Return of $1 Invested")
fig.show()

### Plot Simple Strategy Comparison

In [ ]:
fig = fig_from_df(rets_cum[['ADM','DM','VSMGX']])
fig.update_layout(
    title="Comparison of Strategy Returns",
    xaxis_title="Date",
    yaxis_title="Return of $1 Invested")
fig.show()

### Plot Strategy Comparison with Assets

In [ ]:
fig = fig_from_df(rets_cum[['ADM in SPY','ADM in VSS','ADM in TLT','DM in SPY','DM in VXUS','DM in BND','VSMGX']])
fig.update_layout(
    title="Comparison of Strategy Returns by Asset Invested In At That Time",
    xaxis_title="Date",
    yaxis_title="Return of $1 Invested")
fig.write_image("Acceleration-Dual-Momentum-Long-History.png")
fig.show()

In [ ]:
shorter_lookback = rets_cum[['ADM in SPY','ADM in VSS','ADM in TLT','DM in SPY','DM in VXUS','DM in BND','VSMGX']]
shorter_lookback = shorter_lookback.loc['2020-11-01':]
shorter_lookback = shorter_lookback/shorter_lookback.iloc[0]

fig = fig_from_df(shorter_lookback)
fig.update_layout(
    title="Comparison of Recent Strategy Returns",
    xaxis_title="Date",
    yaxis_title="Return of $1 Invested")
fig.write_image("Acceleration-Dual-Momentum-Short-History.png")
fig.show()